### 一、遞歸神經網路 (Recurrent Neural Network, RNN)

<img src="img/RNN.JPG" alt="drawing" width="600px"/>

- $\bf{\vec{x}}=\left (x_{0}, x_{1}, x_{2},...,x_{t} \right )$

- $\bf{\vec{h}}=\left ( h_{0}, h_{1}, h_{2},...,h_{t} \right )$

#### 1. 單個 RNN  層(別名: 含狀態層、含記憶層)的運算

$$\bf{h}_{t}=tanh\left ( \bf{h}_{t-1}\bf{W}_{h}+\bf{x}_{t}\bf{W}_{x}+ \bf{b} \right )$$

- $\bf{h}_{t}$: 隱藏狀態(hidden state)或隱藏狀態向量(hidden state vector)

#### 2. 往時間方向展開 RNN  層的反向傳播(Backpropagation Through Time, BPTT)

- BPTT 在學習長時間序列資料時的問題:

  (1) BPTT 消耗的電腦記憶體用量隨著時間序列資料的大小成正比增加
  
  (2) 時間愈長，BPTT 的梯度會越不穩定
  
  
- 解決方式: Truncated BPTT

  (1) 處理大型時間序列資料時，以適當長度截斷時間軸方向過長的類神經網路，來建立許多小型的類神經網路，之後用這些區塊為單位(TimeRNN)進行反向傳播學習 (維持正向傳播的連結，只切斷反向傳播的連結)

  [註] RNN 執行 Truncated BPTT 時，必須依照時間順序提供資料來維持正向傳播時間軸上的連結，不可隨機選擇提供資料


<img src="img/TimeRNN.JPG" alt="drawing" width="600px"/>

$$\bf{\vec{x}_{s}}=\left (x_{0}, x_{1}, x_{2},...,x_{T-1} \right ),\ \bf{\vec{h}_{s}}=\left ( h_{0}, h_{1}, h_{2},...,h_{T-1} \right )$$


- Truncated BPTT 的小批次學習: 按照各個批次，移動資料的起始位置

  例子: 1000 個時間序列的輸入資料，以 10 個時間長度為單位進行截斷且小批次的批次數為 2
  
  -> 第一批次起始位置的元素: $x_{0}$、第二批次起始位置的元素: $x_{500}$
  
  第一批次: $(x_{0}, x_{1},...,x_{9})$、$(x_{10}, x_{11},...,x_{19})$、...、$(x_{490}, x_{491},...,x_{499})$
  
  第二批次: $(x_{500}, x_{501},...,x_{509})$、$(x_{510}, x_{511},...,x_{519})$、...、$(x_{990}, x_{991},...,x_{999})$
  
  矩陣表示:
  
  $\begin{bmatrix}
  x_{0},\  x_{1},\ ...\ ,\ x_{9}\\ 
  x_{500},\ x_{501},...,\ x_{509}
  \end{bmatrix}$, 
  $\begin{bmatrix}
  x_{11},\  x_{12},\ ...\ ,\ x_{19}\\ 
  x_{510},\ x_{511},...,\ x_{519}
  \end{bmatrix}$, ... ,
  $\begin{bmatrix}
  x_{490},\ x_{491},...,\ x_{499}\\
  x_{990},\ x_{991},...,\ x_{999}
  \end{bmatrix}$

#### 3. 建立 RNN 層的類別:

In [ ]:
import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx)
        h_next = np.tanh(t)
        
        self.cache(x, h_prex, h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis = 0)
        dWh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt, Wh.T)
        dWx = np.dot(x.T, dt)
        dx = np.dot(dt, Wx.T)
        
        # self.grads[0] => 淺拷貝(shallow copy), self.grads[0][...] => 深拷貝(deep copy) 
        # self.grads[0][...]: 使用三個點[...]固定 Numpy 陣列的記憶體位置並覆寫 Numpy 陣列的元素
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev

#### 4. 建立 TimeRNN 層的類別:

In [ ]:
class TimeRNN:
    
    # stateful = True: 維持 TimeRNN 層的隱藏狀態; stateful = False: 以零矩陣初始化第一個 RNN 層的隱藏狀態
    def __init__(self, Wx, Wh, b, stateful = False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        
        self.h, self.dh = None, None
        
        self.stateful = stateful
    
    # 設定 TimeRNN 層的的隱藏狀態
    def set_state(self, h):
        self.h = h
        
    # 還原 TimeRNN 層的的隱藏狀態
    def reset_state(self):
        self.h = None
        
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape
        
        self.layers = []
        hs = np.empty((N, T, H), dtype = 'f')
        
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H),dtype = 'f')
            
        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)
            
        return hs
    
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape
        
        dhs = np.empty((N, T, D), dtype = 'f')
        dh = 0
        grads = [0, 0, 0]
        
        # 往初始時間的方向累加梯度
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[: ,t , :] + dh)
            dxs[:, t, :] = dx
            
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
        
        # 將梯度覆寫到 self.grads 中
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
            
        self.dh = dh
        
        return dxs

### 二、遞歸神經網路(RNN)的問題與因應的對策

#### 1. RNN 不擅長學習時間序列的長期依存關係

原因: BPTT 可能出現梯度消失或梯度爆炸的問題

<img src="img/BPTT.JPG" alt="drawing" width="500px"/>

直觀的解釋:

- 反向傳播時，tanh 函數微分的數值介於 -1 ~ 1 之間，當通過 tanh 函數 T 次，梯度也會重複減弱 T 次，造成梯度消失

  -> 改進方式: 活化函數使用 ReLU 函數
  

- 若每個 RNN 的權重 $W_{h}$ 皆相同且隱藏狀態變化的傳入值為 dh，則反向傳播通過一系列 Mul 節點後梯度為 $dh\cdot W_{h}^{T}$

  假設 $W_{h}$ 是純量矩陣:
  
  (1) 當 $W_{h}$ > 1, 梯度指數性增加 => 梯度爆炸(exploding gradients)
      
  (2) 當 $W_{h}$ < 1, 梯度指數性減少 => 梯度消失(vanishing gradients)
  
  假設 $W_{h}$ 是非純量矩陣 => 考量矩陣的奇異值是否大於一(必要條件，非充分條件)
  
#### 2. 處理梯度爆炸的對策: 梯度裁減(gradients clipping)

梯度裁減(gradients clipping)演算法: ($\hat{\bf{g}}$ 代表所有類神經網路所使用的參數之梯度整合成一個的結果)

$$if\quad || \hat{\bf{g}} ||\geq \ threshold:$$

$$ \hat{\bf{g}} = \frac{threshold}{|| \hat{\bf{g}} ||}\ \hat{\bf{g}}$$

#### 3. 處理梯度消失的對策: 使用含閘門的 RNN

具代表性的結構:

- 長短期記憶模型(Long short-term memory, LSTM)


- Gated Recurrent Unit (GRU)

### 三、長短期記憶模型(Long short-term memory, LSTM)

#### 1. 單個 LSTM 的計算圖

<img src="img/LSTMcp.JPG" alt="drawing" width="400px"/>

- 輸入值: $\bf{x}$


- 隱藏狀態(hidden state): $\bf{h}$


- 記憶單元(memory cell): $\bf{c}$

  特色: 僅在 LSTM 層內傳遞資料，不會輸出給其他層

[註1] $\sigma$ 代表 sigmoid 函數

[註2] $\odot$: 阿達馬乘積(hadamard product)，兩個矩陣相同位置的元素相乘

**(1) 遺忘閘門(forget gate): f**

$$\bf{f}=\sigma \left (\bf{x_{t}}\bf{W_{x}^{\left ( f \right )}}+ \bf{h_{t-1}}\bf{W_{h}^{\left ( f \right )}}+b^{\left ( f \right )} \right )$$

**說明:** 從上一個時刻的記憶單元中，刪除必須遺忘的資訊 ( $\bf{f} \odot \ \bf{c_{t-1}}$)

**(2) 新的記憶單元: g**

$$\bf{g}=tanh \left (\bf{x_{t}}\bf{W_{x}^{\left ( g \right )}}+ \bf{h_{t-1}}\bf{W_{h}^{\left ( g \right )}}+b^{\left ( g \right )} \right )$$

**說明:** 要在記憶單元($\bf{c_{t}}$)增加的新記憶

**(3) 輸入閘門(input gate): i**

$$\bf{i}=\sigma \left (\bf{x_{t}}\bf{W_{x}^{\left ( i \right )}}+ \bf{h_{t-1}}\bf{W_{h}^{\left ( i \right )}}+b^{\left ( i \right )} \right )$$

**說明:** 判斷新的記憶單元($\bf{g}$)的各個元素是否有當作新資訊的價值，進而取捨要增加的資訊 ($\bf{g}\odot \bf{i}$)

**(4) 輸出閘門(output gate): o**

$$\bf{o}=\sigma \left (\bf{x_{t}}\bf{W_{x}^{\left ( o \right )}}+ \bf{h_{t-1}}\bf{W_{h}^{\left ( o \right )}}+b^{\left ( o \right )} \right )$$

$$\bf{c_{t}}=f\ \odot \ \bf{c_{t-1}} + \bf{g}\  \odot \bf{i}$$

$$\bf{h_{t}}=\bf{o}\ \odot \ tanh\left ( \bf{c_{t}} \right )$$

**說明:** 
 
(i) tanh 函數的輸出可以解釋成某個編碼後資訊的強弱(程度) (作為活化函數使用)

(ii) sigmoid 函數的輸出(0~1)代表的資料通過的比例

(iii) 對 tanh$\left ( \bf{c_{t}} \right )$ 套用輸出閘門 $\bf{o}$ 可解釋為要用 tanh$\left ( \bf{c_{t}} \right )$ 中的各個元素來調整下一個時刻每個隱藏狀態$\left ( \bf{h_{t}} \right )$元素的重要程度

#### 2. 單個 LSTM 的計算圖 (執行權重、偏權值的整合)

<img src="img/affineLSTMcp.JPG" alt="drawing" width="400px"/>

$$\bf{f}=\sigma \left (\bf{x_{t}}\bf{W_{x}^{\left ( f \right )}}+ \bf{h_{t-1}}\bf{W_{h}^{\left ( f \right )}}+b^{\left ( f \right )} \right )$$

$$\bf{g}=tanh \left (\bf{x_{t}}\bf{W_{x}^{\left ( g \right )}}+ \bf{h_{t-1}}\bf{W_{h}^{\left ( g \right )}}+b^{\left ( g \right )} \right )$$

$$\bf{i}=\sigma \left (\bf{x_{t}}\bf{W_{x}^{\left ( i \right )}}+ \bf{h_{t-1}}\bf{W_{h}^{\left ( i \right )}}+b^{\left ( i \right )} \right )$$

$$\bf{o}=\sigma \left (\bf{x_{t}}\bf{W_{x}^{\left ( o \right )}}+ \bf{h_{t-1}}\bf{W_{h}^{\left ( o \right )}}+b^{\left ( o \right )} \right )$$

$$\bf{c_{t}}=f\ \odot \ \bf{c_{t-1}} + \bf{g}\  \odot \bf{i}$$

$$\bf{h_{t}}=\bf{o}\ \odot \ tanh\left ( \bf{c_{t}} \right )$$

**執行權重、偏權值的整合**

當 $\bf{W_{x}}=[\bf{W_{x}^{\left ( f \right )}}, \bf{W_{x}^{\left ( g \right )}}, \bf{W_{x}^{\left ( i \right )}}, \bf{W_{x}^{\left ( o \right )}}]$、$\bf{W_{h}}=[\bf{W_{h}^{\left ( f \right )}}, \bf{W_{h}^{\left ( g \right )}},\bf{W_{h}^{\left ( i \right )}}, \bf{W_{h}^{\left ( o \right )}}]$、$\bf{b}=[\bf{b^{\left ( f \right )}}, \bf{b^{\left ( g \right )}}, \bf{b^{\left ( i \right )}}, \bf{b^{\left ( o \right )}}]$時，

仿射轉換(affine transformation)的統一處理為

$$\bf{x_{t}}\bf{W_{x}} + \bf{h_{t-1}}\bf{W_{h}} + \bf{b}$$

#### 3. TimeLSTM 層的結構

<img src="img/TimeLSTM.JPG" alt="drawing" width="600px"/>

$$\bf{\vec{x}_{s}}=\left (x_{0}, x_{1}, x_{2},...,x_{T-1} \right ),\ \bf{\vec{h}_{s}}=\left ( h_{0}, h_{1}, h_{2},...,h_{T-1} \right )$$

- Truncated BPTT 進行小批次學習

#### 4. 建立 LSTM 層的類別:

In [ ]:
class LSTM:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        
    def forward(self, x, h_prev, c_prev):
        Wx, Wh, b = self.params    
        N, H = h_prev.shape
        
        A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b
        
        # Slice
        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]
        
        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)

        c_next = f * c_prev + g * i
        h_next = o * np.tanh(c_next)
        
        self.cache = (x, h_prev, c_prev, i, f, g, o, c_next)
        
        return h_next, c_next
    
    def backward(self, dh_next, dc_next):
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, c_next = self.cache

        tanh_c_next = np.tanh(c_next)

        ds = dc_next + (dh_next * o) * (1 - tanh_c_next ** 2)

        dc_prev = ds * f

        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i

        di *= i * (1 - i)
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2)

        dA = np.hstack((df, dg, di, do))

        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)

        return dx, dh_prev, dc_prev

#### 5. 建立 TimeLSTM 層的類別:

In [ ]:
class TimeLSTM:
    def __init__(self, Wx, Wh, b, stateful = False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.c = None, None
        self.dh = None
        self.stateful = stateful
    
    def set_state(self, h, c = None):
        self.h, self.c = h, c

    def reset_state(self):
        self.h, self.c = None, None
        
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        H = Wh.shape[0]

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
        
        if not self.stateful or self.c is None:
            self.c = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = LSTM(*self.params)
            self.h, self.c = layer.forward(xs[:, t, :], self.h, self.c)
            hs[:, t, :] = self.h

            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D = Wx.shape[0]

        dxs = np.empty((N, T, D), dtype='f')
        dh, dc = 0, 0
        grads = [0, 0, 0]
        
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh, dc = layer.backward(dhs[:, t, :] + dh, dc)
            dxs[:, t, :] = dx
            
            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        
        self.dh = dh
        
        return dxs

### 四、Gated Recurrent Unit (GRU)

#### 1. 單個 GRU 的計算圖:

<img src="img/GRU.JPG" alt="drawing" width="400px"/>

- 輸入值: $\bf{x}$


- 隱藏狀態(hidden state): $\bf{h}$


[註1] $\sigma$ 代表 sigmoid 函數

[註2] $\odot$: 阿達馬乘積(hadamard product)，兩個矩陣相同位置的元素相乘

**(1) 重置閘門: r**

$$\bf{r}=\sigma \left (\bf{x_{t}}\bf{W_{x}^{\left ( r \right )}}+ \bf{h_{t-1}}\bf{W_{h}^{\left ( r \right )}}+b^{\left ( r \right )} \right )$$

**說明:** 用來決定"忽略"過去隱藏狀態的程度

**(2) 更新閘門: z**

$$\bf{z}=\sigma \left (\bf{x_{t}}\bf{W_{x}^{\left ( z \right )}}+\bf{h_{t-1}}\bf{W_{h}^{\left ( z \right )}}+b^{\left ( z \right )} \right )$$

$$\bf{\tilde{h}}=tanh\left [\bf{x_{t}}\bf{W_{x}} + \left (\bf{r} \odot \bf{h_{t-1}} \right )\bf{W_{h}} + b \right ]$$

$$\bf{h_{t}}=\left (1-z \right ) \odot \bf{h_{t-1}} + \bf{z} \odot \bf{\tilde{h}}$$

**說明:** 用來更新隱藏狀態的閘門，相當於 LSTM 的遺忘閘門與輸入閘門。$\left (1-\bf{z} \right ) \odot \bf{h_{t-1}}$ 代表刪除過去隱藏狀態所有必須要忘記的資料; $\bf{z} \odot \bf{\tilde{h}}$ 代表輸入閘門的權重加到新增的資料中。